In [1]:
import time
import numpy as np

In [2]:
from bokeh.models.sources import ColumnDataSource
from bokeh.plotting import figure
from bokeh.io import output_notebook, show, push_notebook
from bokeh.models import DatetimeTickFormatter
from math import pi

In [3]:
from elasticsearch import Elasticsearch
from elasticsearch_dsl import Search, Q
from pandasticsearch import Select, DataFrame

In [4]:
import datetime
from pytz import timezone
f"{datetime.datetime.now(timezone('US/Central')):%Y-%m-%d %H:%M:%S}"

'2019-07-31 09:20:10'

Using the new query format

In [5]:
hostname="http://noname-sms.us.cray.com:30200"
client =  Elasticsearch(hostname, http_compress=True)

gte="2019-07-29 00:00:00"
lte="2019-07-29 00:01:00"

q="""
    {
        "size":0,
        "query": {
            "bool": {
              "must": [{
                  "match_all": {}
                },
                {
                  "range": {
                    "timereported": {
                      "gte": "%s",
                      "lte": "%s",
                      "format": "yyyy-MM-dd HH:mm:ss||yyyy-MM-dd||epoch_millis"
                }
              }
            }
          ],
          "must_not": []
        }
      },
        "_source":{
            "excludes":[]
        },
        "aggs": {
        "2": {
          "date_histogram": {
            "field": "timereported",
            "interval": "30s",
            "time_zone": "America/Chicago",
            "min_doc_count": 1
          }
        }
      }
    }
""" % (gte,lte)

#print("Query: \"%s\"" % q)
resp = client.search(index="shasta-logs-*", body=q)
print("Number of responses: " + "{:,}".format(resp['hits']['total']))

Number of responses: 0


In [6]:
output_notebook()

Loading BokehJS ...

In [7]:
p = figure(plot_width=800, plot_height=400)
test_data = ColumnDataSource(data=dict(x=[0], y=[0]))
#line = my_figure.line("x", "y", source=test_data)
line = p.circle("x", "y", source=test_data, size = 8, color = 'navy', alpha=0.3)
p.title.text = 'Message Counts per 30 minutes'
p.background_fill_color="#f5f5f5"
p.grid.grid_line_color="white"
p.yaxis.axis_label = 'Count'
p.xaxis.axis_label =' timereported per 30 minutes'
p.xaxis.formatter=DatetimeTickFormatter(
        hours=["%d %B %Y"],
        days=["%d %B %Y"],
        months=["%d %B %Y"],
        years=["%d %B %Y"],
    )
p.xaxis.major_label_orientation = pi/4
handle = show(p, notebook_handle=True)

In [8]:
from threading import Thread

stop_threads = False

def blocking_callback(id, stop):
    new_data=dict(x=[0], y=[0])

    step = 0
    step_size = 0.1  # increment for increasing step
    max_step = 10  # arbitrary stop point for example
    period = 1  # in seconds (simulate waiting for new data)
    n_show = 1000  # number of points to keep and show

    while True:

        new_data['x'] = [step]
        new_data['y'] = [np.random.randint(0,1000)]

        test_data.stream(new_data, n_show)

        push_notebook(handle=handle)
        step += step_size
        time.sleep(period)

        if stop():
            print("exit")
            break

thread = Thread(target=blocking_callback, args=(id, lambda: stop_threads))
thread.start()

In [ ]:
# preceding streaming is not blocking
for cnt in range(10):
    print("Do this, while plot is still streaming", cnt)

In [ ]:
# you might also want to stop the thread
stop_threads=True
del thread